<a href="https://colab.research.google.com/github/ShaliniR8/google-colab-notebooks/blob/master/SimpleRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DATASET FROM http://jmcauley.ucsd.edu/data/amazon/**\
Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering\
*R. He, J. McAuley\
WWW, 2016* 

In [ ]:
path = r'C:\Users\KIIT\Downloads\ratings_Digital_Music.csv'

**Pandas data frame**\
This code reads the data into a pandas data frame:

In [ ]:
import pandas as pd

df = pd.read_csv(path)
df.columns = ['customer_id', 'product_id', 'rating','timestamp']

In [ ]:
df.head()

,customer_id,product_id,rating,timestamp
0,A1WR23ER5HMAA9,5555991584,5.0,953424000
1,A2IR4Q0GPAFJKW,5555991584,4.0,1393545600
2,A2V0KUVAB9HSYO,5555991584,4.0,966124800
3,A1J0GL9HCA7ELW,5555991584,5.0,1007683200
4,A3EBHHCZO6V2A4,5555991584,5.0,1158019200


**SIMPLE RECOMMENDER USING JACCARD SIMILARITY**

In [ ]:
from collections import defaultdict

usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
products = set()

In [ ]:
for d in range(len(df)):
    user = df['customer_id'][d]
    product = df['product_id'][d]
    usersPerItem[product].add(user)
    itemsPerUser[user].add(product)
    products.add(product)

In [ ]:
def jaccard(u1, u2):
    
    
   
    num = len(u1.intersection(u2))
    denom = len(u1.union(u2))
  
    return num/denom

def mostSimilarItems(i):
    
    users = usersPerItem[i]
    items = []
    candidateItems = set()
     
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    
    for j in candidateItems:
        if i==j:
            continue
        
        sim = jaccard(users, usersPerItem[j])
        items.append((sim,j))
        
    items.sort( reverse= True )
    return items[:10]
    
    
    

In [ ]:
x = mostSimilarItems( df['product_id'][10] )

**SIMPLE RECOMMENDER USING COLLABORATIVE FILTERING**

In [ ]:
reviewsPerItem = defaultdict(set)
reviewsPerUser = defaultdict(set)



for d in range(len(df)):
    user = df['customer_id'][d]
    item = df['product_id'][d]
    
    reviewsPerUser[user].add((df['rating'][d],df['customer_id'][d],df['product_id'][d]))
    reviewsPerItem[item].add((df['rating'][d],df['customer_id'][d],df['product_id'][d]))
    
    
    

In [ ]:
import numpy as np
Mean = np.mean(df['rating'])
Mean

4.5402491611892275

In [ ]:
def predictRating(user, item):
    
    ratings = []
    similarity = []
    for r in reviewsPerUser[user]:
        i2 = r[2]
        if item == i2:
            continue
        ratings.append(r[0])
        similarity.append(jaccard(usersPerItem[i2], usersPerItem[item]))
    if sum(similarity) > 0:
        w = [ r*s for r,s in  zip(ratings, similarity)]
        return sum(w)/sum(similarity)
    
    else:
        return Mean

In [ ]:


predictRating(df['customer_id'][19], df['product_id'][9])


5.0